In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [4]:
data = pd.read_csv('data_stocks.csv')
data.head()


,DATE,SP500,NASDAQ.AAL,NASDAQ.AAPL,NASDAQ.ADBE,NASDAQ.ADI,NASDAQ.ADP,NASDAQ.ADSK,NASDAQ.AKAM,NASDAQ.ALXN,...,NYSE.WYN,NYSE.XEC,NYSE.XEL,NYSE.XL,NYSE.XOM,NYSE.XRX,NYSE.XYL,NYSE.YUM,NYSE.ZBH,NYSE.ZTS
0,1491226200,2363.6101,42.3300,143.6800,129.6300,82.040,102.2300,85.2200,59.760,121.52,...,84.370,119.035,44.40,39.88,82.03,7.36,50.22,63.86,122.000,53.350
1,1491226260,2364.1001,42.3600,143.7000,130.3200,82.080,102.1400,85.6500,59.840,121.48,...,84.370,119.035,44.11,39.88,82.03,7.38,50.22,63.74,121.770,53.350
2,1491226320,2362.6799,42.3100,143.6901,130.2250,82.030,102.2125,85.5100,59.795,121.93,...,84.585,119.260,44.09,39.98,82.02,7.36,50.12,63.75,121.700,53.365
3,1491226380,2364.3101,42.3700,143.6400,130.0729,82.000,102.1400,85.4872,59.620,121.44,...,84.460,119.260,44.25,39.99,82.02,7.35,50.16,63.88,121.700,53.380
4,1491226440,2364.8501,42.5378,143.6600,129.8800,82.035,102.0600,85.7001,59.620,121.60,...,84.470,119.610,44.11,39.96,82.03,7.36,50.20,63.91,121.695,53.240


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41266 entries, 0 to 41265
Columns: 502 entries, DATE to NYSE.ZTS
dtypes: float64(501), int64(1)
memory usage: 158.0 MB


In [6]:
#Exercise 1:

df = data.drop(['DATE', 'SP500'], axis=1)

In [7]:
cor = df.corr()

In [8]:
corr = pd.DataFrame(cor, columns=cor.index, index = cor.index)

In [9]:
corr.stack().drop_duplicates().sort_values(ascending=False)[ :20]

NASDAQ.AAL    NASDAQ.AAL      1.000000
NASDAQ.GOOG   NASDAQ.GOOGL    0.998352
NASDAQ.DISCA  NASDAQ.DISCK    0.996855
NASDAQ.NWS    NASDAQ.NWSA     0.996612
NASDAQ.FOX    NASDAQ.FOXA     0.996182
NYSE.DD       NYSE.DOW        0.986025
NYSE.UA       NYSE.UAA        0.983613
NYSE.DVN      NYSE.MRO        0.983269
NYSE.CMS      NYSE.ES         0.981819
              NYSE.DTE        0.981701
NYSE.LMT      NYSE.RTN        0.981134
NYSE.MA       NYSE.V          0.980574
NYSE.DVN      NYSE.RIG        0.980150
NASDAQ.ADBE   NYSE.MA         0.979629
NASDAQ.ETFC   NYSE.BK         0.978342
NYSE.DTE      NYSE.ES         0.978332
NASDAQ.NVDA   NYSE.AON        0.977920
NYSE.MCD      NYSE.YUM        0.977900
NYSE.AEP      NYSE.DTE        0.977853
NASDAQ.PYPL   NYSE.MA         0.977071
dtype: float64

In [ ]:
# Exercise 2

from math import sqrt
from scipy.cluster.vq import kmeans,vq
from sklearn.cluster import KMeans

returns = df.pct_change().mean()*252
returns = pd.DataFrame(returns)
returns.columns = ['Returns']
returns['Volatility'] = df.pct_change().std()*sqrt(252)
returns.head()

In [ ]:
dta = np.asarray([np.asarray(returns.Returns), np.asarray(returns.Volatility)]).T
dta.shape

X = dta
distortions =[]
for k in range(2,20):
    k_means = KMeans(n_clusters = k)
    k_means.fit(X)
    distortions.append(k_means.inertia_)

In [ ]:
centroids,_= kmeans(dta, 5)
# Assign each sample to a cluster
idx,_ = vq(dta, centroids)

In [ ]:
from pylab import plot, show
plot(dta[idx==0,0],dta[idx==0,1],'ob',
     dta[idx==1,0],dta[idx==1,1],'oy',
     dta[idx==2,0],dta[idx==2,1],'or',
     dta[idx==3,0],dta[idx==3,1],'og',
     dta[idx==4,0],dta[idx==4,1],'om')
plot(centroids[:,0],centroids[:,1],'sg',markersize=8)
show()

In [ ]:
#identify the outlier
print(returns.idxmax())

returns.drop(['NYSE.XRX'],inplace=True)

dta = np.asarray([np.asarray(returns.Returns), np.asarray(returns.Volatility)]).T
centroids,_= kmeans(dta, 5)
# Assign each sample to a cluster
idx,_ = vq(dta, centroids)

# some plotting using numpy's logical indexing
plot(dta[idx==0,0],dta[idx==0,1],'ob',
     dta[idx==1,0],dta[idx==1,1],'oy',
     dta[idx==2,0],dta[idx==2,1],'or',
     dta[idx==3,0],dta[idx==3,1],'og',
     dta[idx==4,0],dta[idx==4,1],'om')
plot(centroids[:,0],centroids[:,1],'sg',markersize=8)
show()

In [ ]:
details = [(name,cluster) for name, cluster in zip(returns.index,idx)]
details[:5]

In [ ]:
df1 = pd.DataFrame(details, columns=['Stock_Name','Cluster_No'])
df1.Cluster_No.value_counts()

In [ ]:
import seaborn as sns
ind = df1.Cluster_No.value_counts()
plt.figure(figsize=(12,4))
sns.barplot(x=ind.index, y=ind.values, alpha=0.6)
plt.ylabel('Number of Occurences',fontsize=12)
plt.xlabel('Cluster Number',fontsize=12)
plt.title('Barplot - Indicator',fontsize =16)
plt.show()

In [ ]:
# Exercise 3:

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df.values

sc = StandardScaler().fit_transform(X)
pca = PCA(n_components=150).fit(sc)

var = pca.explained_variance_ratio_

# cumulative varince explains
var1 = np.cumsum(np.round(var, decimals=4))
print(var1)

In [ ]:
plt.plot(var1)

In [ ]:
X_scaled = PCA(n_components=25).fit(sc)
comp = pd.DataFrame(pca.components_, columns = df.columns)
comp.head()